# Technical Analysis Template: sRoC = WMA + RoC

Follow the trend the using smooth Rate of Change. The smooth function is WMA.

In [1]:
import qnt.data as qndata
import qnt.stats as qnstats
import qnt.xr_talib as qnxrtalib
import qnt.graph as qngraph
import qnt.forward_looking as qnfl

# Data

In [2]:
data = qndata.load_data(
    min_date="2014-01-01", 
    max_date="2019-08-28", # final calculations should not limit max_date
    forward_order=True, 
    dims=("time", "field", "asset")
)

fetched chunk 1/10 4s
fetched chunk 2/10 56s
fetched chunk 3/10 71s
fetched chunk 4/10 75s
fetched chunk 5/10 79s
fetched chunk 6/10 83s
fetched chunk 7/10 111s
fetched chunk 8/10 116s
fetched chunk 9/10 120s
fetched chunk 10/10 123s
Data loaded 123s


# Calc output

In [3]:
SROC_POSITIVE_TREND_LEVEL = 0.0125

wma = qnxrtalib.WMA(data.sel(field='close'), 290)
sroc = qnxrtalib.ROCP(wma, 35)

is_liquid = data.sel(field="is_liquid")
weights = is_liquid.where(sroc > SROC_POSITIVE_TREND_LEVEL)

weights = weights / weights.sum("asset", skipna=True)

output = weights.fillna(0.0)

# Stats

In [4]:
stat = qnstats.calc_stat(data, output)
display(stat.to_pandas().tail())

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover
time,,,,,,,,,,
2019-08-22,1.336233,-0.000662,0.109256,-0.012514,0.172493,1.001919,0.109465,1.0,937.0,0.038981
2019-08-23,1.312917,-0.017449,0.109742,-0.029745,0.172493,0.938353,0.102977,1.0,937.0,0.038975
2019-08-26,1.323573,0.008116,0.109828,-0.021870,0.172493,0.958123,0.105229,1.0,937.0,0.038975
2019-08-27,1.322865,-0.000535,0.109703,-0.022393,0.172493,0.986816,0.108257,1.0,937.0,0.038974
2019-08-28,1.326223,0.002538,0.109710,-0.019912,0.172493,0.993914,0.109042,1.0,937.0,0.039021


# Improvement

In [9]:
stat_per_asset = qnstats.calc_stat(data, output, max_periods=20, per_asset = True)
improved_output = output.where(
    stat_per_asset.sel(field='sharpe_ratio').rolling(time=20).min() > -12
)
stat = qnstats.calc_stat(data, improved_output, max_periods=252 * 3)
stat.to_pandas().tail()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover
time,,,,,,,,,,
2019-08-22,1.322140,-0.000568,0.107553,-0.011732,0.16732,0.964483,0.103733,1.0,849.0,0.035540
2019-08-23,1.298576,-0.017822,0.108066,-0.029346,0.16732,0.898783,0.097128,1.0,849.0,0.035543
2019-08-26,1.309464,0.008385,0.108163,-0.021207,0.16732,0.921686,0.099692,1.0,849.0,0.035550
2019-08-27,1.309161,-0.000231,0.108091,-0.021433,0.16732,0.943487,0.101983,1.0,849.0,0.035575
2019-08-28,1.312511,0.002558,0.108097,-0.018930,0.16732,0.953567,0.103078,1.0,849.0,0.035618


# Checks

In [6]:
# Use the function from 'qnfl' ensures that no forward-looking
# is taking place. 
def strategy():
    """
    it is the same strtegy, but implemented with xarray
    Entire code of strategy calculation is collected here.
    """
    data = qndata.load_data(
        min_date="2014-01-01", 
        # max_date="2019-08-28", # final calculations should not limit max_date
        forward_order=True, 
        dims=("time", "field", "asset")
    )

    SROC_POSITIVE_TREND_LEVEL = 0.0125

    wma = qnxrtalib.WMA(data.sel(field='close'), 290)
    sroc = qnxrtalib.ROCP(wma, 35)

    is_liquid = data.sel(field="is_liquid")
    weights = is_liquid.where(sroc > SROC_POSITIVE_TREND_LEVEL)

    weights = weights / weights.sum("asset", skipna=True)

    output = weights.fillna(0.0)
    
    stat_per_asset = qnstats.calc_stat(data, output, max_periods=20, per_asset = True)
    improved_output = output.where(
        stat_per_asset.sel(field='sharpe_ratio').rolling(time=20).min() > -10
    )
    
    return improved_output

# This function runs strategy twice on the different periods: 
# the entire data and data the with a cropped last half year.
# After that this function compares outputs. 
# Overlapped outputs must be same.
output_final = qnfl.load_data_calc_output_and_check_forward_looking(strategy)

Computing of the whole output...
fetched chunk 1/10 23s
fetched chunk 2/10 27s
fetched chunk 3/10 31s
fetched chunk 4/10 35s
fetched chunk 5/10 40s
fetched chunk 6/10 44s
fetched chunk 7/10 49s
fetched chunk 8/10 54s
fetched chunk 9/10 58s
fetched chunk 10/10 61s
Data loaded 61s
Computing of the cropped output...
fetched chunk 1/9 5s
fetched chunk 2/9 9s
fetched chunk 3/9 13s
fetched chunk 4/9 18s
fetched chunk 5/9 23s
fetched chunk 6/9 27s
fetched chunk 7/9 32s
fetched chunk 8/9 37s
fetched chunk 9/9 39s
Data loaded 39s
Ok. There is no forward looking.


In [7]:
# correlation check
# your strategy should not correlate with other strategies before submission
qnstats.print_correlation(output_final, data)


The number of systems with a larger Sharpe ratio and correlation larger than 0.8: 16
The max correlation value (with systems with a larger Sharpe ratio): 0.9799380659211977
Current sharpe ratio(3y): 0.9315966554574596



# Save output

In [8]:
qndata.write_output(output_final)

write output: /home/username/fractions.nc.gz
